# Websockets 

In [ ]:
import sys
import asyncio
import websockets
import json
from PyQt5.QtWidgets import QApplication, QWidget, QVBoxLayout, QLabel
from PyQt5.QtCore import QThread, pyqtSignal


class WebSocketThread(QThread):
    """Handles WebSocket connection in a separate thread to avoid freezing the UI."""
    telemetry_received = pyqtSignal(str, str, str)  # Emit status, battery, latency

    async def receive_telemetry(self):
        """Connects to the WebSocket server and receives telemetry data asynchronously."""
        uri = "ws://192.168.2.2:5003"  # TI board's IP address

        try:
            async with websockets.connect(uri) as websocket:
                print(f"Connected to {uri}")

                while True:
                    message = await websocket.recv()
                    data = json.loads(message)

                    latency = data.get("latency", 0)
                    battery = data.get("battery", 0)
                    status = data.get("status", "N/A")

                    self.telemetry_received.emit(status, f"{battery:.1f}%", f"{latency:.2f} ms")  # Send to UI

        except websockets.ConnectionClosed:
            print("Connection closed")
        except Exception as e:
            print(f"WebSocket error: {e}")

    def run(self):
        """Runs the WebSocket event loop inside QThread."""
        loop = asyncio.new_event_loop()
        asyncio.set_event_loop(loop)
        loop.run_until_complete(self.receive_telemetry())


class TelemetryApp(QWidget):
    def __init__(self):
        super().__init__()
        self.initUI()

        # Start WebSocket thread
        self.websocket_thread = WebSocketThread()
        self.websocket_thread.telemetry_received.connect(self.update_telemetry)
        self.websocket_thread.start()

    def initUI(self):
        layout = QVBoxLayout()

        self.status_label = QLabel("Status: N/A", self)
        self.battery_label = QLabel("Battery: N/A", self)
        self.latency_label = QLabel("Latency: N/A", self)

        layout.addWidget(self.status_label)
        layout.addWidget(self.battery_label)
        layout.addWidget(self.latency_label)

        self.setLayout(layout)
        self.setWindowTitle("Telemetry Monitor")
        self.resize(300, 200)

    def update_telemetry(self, status, battery, latency):
        """Update telemetry labels in the UI."""
        self.status_label.setText(f"Status: {status}")
        self.battery_label.setText(f"Battery: {battery}")
        self.latency_label.setText(f"Latency: {latency}")

        print(f"Updated UI -> Status: {status}, Battery: {battery}, Latency: {latency}")  # Debugging


if __name__ == "__main__":
    app = QApplication(sys.argv)
    window = TelemetryApp()
    window.show()
    sys.exit(app.exec())


QSocketNotifier: Can only be used with threads started with QThread


SystemExit: 0

/home/davidsober/.local/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3587: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


WebSocket error: timed out during opening handshake


chat explanation of uri vs url 
Feature	URI (Uniform Resource Identifier)	URL (Uniform Resource Locator)
Definition	A general term for identifying a resource.	A specific type of URI that provides a way to locate a resource.

In [ ]:
class WebSocketThread(QThread):
    """Handles WebSocket connection in a separate thread to avoid freezing the UI."""
    telemetry_received = pyqtSignal(str, str, str)  # Emit status, battery, latency



class inherits from QThread which is he Qt's threading module 
Here is what chat says when comparing QThread vs multiprocesing 
1. QThread (Multithreading)

    Uses threads, not processes, meaning all threads share the same memory space.
    Lower overhead because creating and managing threads is typically faster than creating new processes.
    Less CPU usage in many cases because context switching between threads is cheaper than switching between processes.
    Limited by Python's GIL (Global Interpreter Lock) if you’re doing CPU-bound tasks in Python, but if your work is mostly I/O-bound (e.g., network requests, file handling, GUI updates), QThread is often the better choice.

2. Multiprocessing (New Processes)

    Creates separate processes, which do not share memory directly.
    Higher CPU overhead because each process has its own memory space and needs inter-process communication (IPC) mechanisms to share data.
    Bypasses Python’s GIL, making it better suited for CPU-intensive tasks (e.g., heavy computations, number crunching).
    More system resource consumption since each process needs separate memory allocation

Conclusion: on pc app it does not matter much since the pc can handle either. on Ti side it does matter since it has limited power. we might need to look into proper concurrency on Ti side with C (most likely the case so run some tests later) 

pyqtSignal: 
allows for comms between objects in a thread safe manner. sounds like the multiprocessing queue
They are the same high level Idea but multiprocessing is for processes ofc and qthread is for qt gui based threads. course

In [ ]:
    async def receive_telemetry(self):
        """Connects to the WebSocket server and receives telemetry data asynchronously."""
        uri = "ws://192.168.2.2:5003"  # TI board's IP address

        try:
            async with websockets.connect(uri) as websocket:
                print(f"Connected to {uri}")


async
in py is part of std lib. it needs an await keyword so it knows what to wait for

asyncio provides cooperative multitasking: tasks yield control when they await, allowing other tasks to run.
QThread provides preemptive multitasking: the OS schedules threads separately.
If you’re using Qt, asyncio can conflict with the Qt event loop. You may need QEventLoop or asyncqt to integrate them.

- 

- async def receive_telemetry(self): This is an asynchronous method that will handle the WebSocket communication.
- uri = "ws://192.168.2.2:5003": This is the WebSocket URI (the address of the TI board’s WebSocket server). Change it to your device's IP if needed.
- async with websockets.connect(uri) as websocket:: This establishes a WebSocket connection asynchronously. The async with ensures that the connection is properly closed after the code block finishes executing.
- print(f"Connected to {uri}"): A simple log message indicating that the connection has been successfully established.

Looking at the async with x as y 

x as y
- y is what x.++aenter++() returns so it could be different than x
- 
turns out it is about proper cleanup 

In [ ]:
async with websockets.connect(uri) as websocket:
    print(f"Connected to {uri}")

##### is equivalent to:

conn = await websockets.connect(uri)
try:
    websocket = conn  # Assign alias
    print(f"Connected to {uri}")
finally:
    await conn.__aexit__(None, None, None)  # Ensure cleanup

ok under the hood explanation 
What happens under the hood:

    websockets.connect(uri) creates a WebSocket connection object.
    Python calls connection.__aenter__(), which:
        Waits for the connection to be established (await).
        Returns the active WebSocket object (websocket).
    Inside the block, websocket is now ready to use.
    When the block exits, Python calls connection.__aexit__(), which:
        Closes the WebSocket properly (await ensures async cleanup).

# next section

In [ ]:
                while True:
                    message = await websocket.recv()
                    data = json.loads(message)

                    latency = data.get("latency", 0)
                    battery = data.get("battery", 0)
                    status = data.get("status", "N/A")

                    self.telemetry_received.emit(status, f"{battery:.1f}%", f"{latency:.2f} ms")  # Send to UI


the args in the data.get() are default vals in case no data is received 

await 
- await pauses execution of the coroutine until the awaited task is done, without blocking the entire program.
- ok so we have a coroutine runing and that part pauses. so it is sort of like having a pause in a diff process. only that process gets paused while the other continue. nonblocking pausing basically. 

No. Each await statement only waits for one specific async task.

    await websocket.recv() only waits for the next WebSocket message.

- the loop starts and hits await websocket.recv().
- await pauses execution of the function until a message is received.
- Meanwhile, other async tasks can run (e.g., UI updates, network events).
- When the WebSocket receives a message, await resumes execution.
- The message is converted into JSON, and telemetry_received.emit() sends data to the UI.
- Loop repeats → waits again at await websocket.recv().





In [ ]:
self.telemetry_received.emit(status, f"{battery:.1f}%", f"{latency:.2f} ms")  # Send to UI

Understanding emit in PyQt

- Signals and Slots: Qt uses a signal and slot mechanism to handle communication between objects. A signal is emitted when a particular event occurs, and a slot is a function that responds to that signal.

- emit: The emit method is used to trigger the signal, passing the necessary arguments to any connected slots. In your code, self.telemetry_received.emit(...) sends the status, battery, and latency data to any function connected to the telemetry_received sign

- ok all this to saw that the telemetry_received var was defined as a signal here 
    - telemetry_received = pyqtSignal(str, str, str)  # Emit status, battery, latency
- and anything can emit to this signal as long as it has the 3 args
    - that signal then receives the data emitted then it can do whatever with it like put it on the gui

# to be clear regarding emit, pyqtsignal and connect
- defining the signal is here
    - telemetry_received = pyqtSignal(str, str, str)  # Defines the signal
- emitting the signal is her e
    - self.telemetry_received.emit(status, f"{battery:.1f}%", f"{latency:.2f} ms")  # Send to UI
- receiving the signal is here 
    - self.websocket_thread.telemetry_received.connect(self.update_telemetry)



In [ ]:
class TelemetryApp(QWidget):
    def __init__(self):
        super().__init__()
        self.initUI()

        # Start WebSocket thread
        self.websocket_thread = WebSocketThread()
        self.websocket_thread.telemetry_received.connect(self.update_telemetry)
        self.websocket_thread.start()


- class TelemetryApp(QWidget): This is the main GUI application class, which inherits from QWidget (the base class for all PyQt widgets).
- self.initUI(): This method initializes the GUI components.
= self.websocket_thread = WebSocketThread(): Creates an instance of the WebSocketThread class that will handle the WebSocket communication in a separate thread.
= self.websocket_thread.telemetry_received.connect(self.update_telemetry): Connects the telemetry_received signal from the WebSocket thread to the update_telemetry method. This means every time the thread emits new telemetry data, update_telemetry will be called.
- self.websocket_thread.start(): Starts the WebSocket thread.



In [ ]:
    def initUI(self):
        layout = QVBoxLayout()

        self.status_label = QLabel("Status: N/A", self)
        self.battery_label = QLabel("Battery: N/A", self)
        self.latency_label = QLabel("Latency: N/A", self)

        layout.addWidget(self.status_label)
        layout.addWidget(self.battery_label)
        layout.addWidget(self.latency_label)

        self.setLayout(layout)
        self.setWindowTitle("Telemetry Monitor")
        self.resize(300, 200)


sets up the layout and the widgets we use

In [ ]:
    def update_telemetry(self, status, battery, latency):
        """Update telemetry labels in the UI."""
        self.status_label.setText(f"Status: {status}")
        self.battery_label.setText(f"Battery: {battery}")
        self.latency_label.setText(f"Latency: {latency}")

        print(f"Updated UI -> Status: {status}, Battery: {battery}, Latency: {latency}")  # Debugging


the function above is connected to telemetry_received from this line below in the contructor of TelemetryApp
- self.websocket_thread.telemetry_received.connect(self.update_telemetry)



# Why use async inside a thread? 

1. async is needed for WebSockets in this case because:
    - The websockets library is designed to be async-first and requires an asyncio event loop.
    websockets.connect() and websocket.recv() must be awaited inside an async function.

2. A QThread runs its run() method in a separate thread, so it won’t block the main thread.
    - But QThread itself is not async. It just runs synchronously in another thread

ok so we need async behaviour but threading does not provide this